In [1]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

import toml
import sys
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModel, AutoTokenizer
import random
import accelerate
from tqdm import tqdm

!export WANDB_NOTEBOOK_NAME=minimum_sample_Bio.py

In [38]:
%load_ext autoreload
%autoreload 2

from utils.utils import *
from train import training_loop_
from utils.collate import MultiencoderTokenizedDataset, TokenizedCollator
from utils.model_utils import get_sentence_embedding_dimension, load_encoder
from utils.streaming_utils import load_streaming_embeddings, process_batch
from translators.Discriminator import Discriminator
from torch.optim.lr_scheduler import LambdaLR
from utils.gan import LeastSquaresGAN, RelativisticGAN, VanillaGAN
from utils.eval_utils import EarlyStopper, eval_loop_
from utils.wandb_logger import Logger
from datasets import Dataset

from patient_data import load_tcga_patient_data

sys.path.append("prep")

from base import GeneDrugVAE
from VAE_GDSC import GDSCDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
cfg_dict = toml.load(f"configs/unsupervised.toml")
cfg = SimpleNamespace(**{k: v for d in cfg_dict.values() for k, v in d.items()})
# cfg.num_points = 256
cfg.bs = 256
cfg.epochs = 50
use_val_set = hasattr(cfg, "val_size")
cfg.wandb_project = "bio"
cfg.wandb_name = "test_bio"
cfg.sup_emb = "cellline_exp"
cfg.unsup_emb = "tcga_exp"
cfg.val_size = 0.2
cfg

namespace(seed=5,
          sampling_seed=5,
          train_dataset_seed=10,
          val_dataset_seed=42,
          normalize_embeddings=True,
          mixed_precision='fp16',
          weight_init='kaiming',
          dataset='nq',
          max_seq_length=64,
          unsup_emb='tcga_exp',
          sup_emb='cellline_exp',
          n_embs_per_batch=1,
          finetune_mode=False,
          noise_level=0.0,
          style='res_mlp',
          norm_style='batch',
          depth=3,
          transform_depth=4,
          d_adapter=1024,
          d_hidden=1024,
          d_transform=1024,
          use_small_output_adapters=False,
          use_residual_adapters=True,
          gan_style='least_squares',
          disc_depth=5,
          disc_dim=1024,
          use_residual=True,
          bs=256,
          gradient_accumulation_steps=1,
          lr=2e-05,
          no_scheduler=True,
          max_grad_norm=1000.0,
          loss_coefficient_reverse_rec=0.0,
          loss_c

In [40]:
random.seed(cfg.seed)
torch.manual_seed(cfg.seed)
np.random.seed(cfg.seed)
torch.cuda.manual_seed(cfg.seed)

# Patient data

In [41]:
df, _ = load_tcga_patient_data()
drugs = df["SMILES"]
exp = df.iloc[:, 6:-2]
gene_exp = pd.concat([df["cases.submitter_id"], exp], axis=1).drop_duplicates()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "seyonec/ChemBERTa-zinc-base-v1"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model = model.to(device)
smiles_list = df["SMILES"].unique().tolist()
inputs = tokenizer(smiles_list, padding=True, truncation=True, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]

drugs = pd.DataFrame(embeddings.cpu().numpy())
drugs.index = smiles_list

In [42]:
gene_exp_values = (
    gene_exp.iloc[:, 1:].astype("float32").set_index(gene_exp["cases.submitter_id"])
)

pairs = [
    (gene_exp_values.loc[patient].values, drugs.loc[drug].values)
    for _, (patient, drug) in df[["cases.submitter_id", "SMILES"]].iterrows()
    if patient in gene_exp_values.index and drug in drugs.index
]

if pairs:
    gene_list, drug_list = zip(*pairs)
    x_gene_tensor = torch.tensor(gene_list, dtype=torch.float32, device=device)
    x_drug_tensor = torch.tensor(drug_list, dtype=torch.float32, device=device)
else:
    x_gene_tensor = torch.empty(0, dtype=torch.float32, device=device)
    x_drug_tensor = torch.empty(0, dtype=torch.float32, device=device)

In [43]:
gene_dim = x_gene_tensor.shape[1]
smiles_dim = x_drug_tensor.shape[1]
latent_dim = 128

In [44]:
patient_vae = GeneDrugVAE(
    gene_dim, drug_dim=smiles_dim, proj_dim=256, hidden=512, latent=128
).to(device)
state = torch.load("prep/models/tcga/tcga_best.pt", map_location=device)
patient_vae.load_state_dict(state)
patient_vae.eval()

GeneDrugVAE(
  (g1): Linear(in_features=6255, out_features=512, bias=True)
  (g2): Linear(in_features=512, out_features=256, bias=True)
  (p1): Linear(in_features=768, out_features=256, bias=True)
  (p2): Linear(in_features=256, out_features=256, bias=True)
  (fc): Linear(in_features=512, out_features=512, bias=True)
  (mu): Linear(in_features=512, out_features=128, bias=True)
  (lv): Linear(in_features=512, out_features=128, bias=True)
  (d1): Linear(in_features=128, out_features=256, bias=True)
  (d2): Linear(in_features=256, out_features=6255, bias=True)
  (head): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=128, out_features=1, bias=True)
  )
)

# Cell Line data

In [45]:
cellline = pd.read_csv(
    "../UniBridge-DR/dataset/gdsc/cellline_info.csv.gz",
    usecols=["COSMIC_ID", "SMILES", "NAME", "Z_score"],
)
exp = pd.read_csv("../UniBridge-DR/dataset/gdsc/exp.csv.gz", index_col=0)
smiles_list = cellline["SMILES"].unique().tolist()

inputs = tokenizer(smiles_list, padding=True, truncation=True, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]

drugs = pd.DataFrame(embeddings.cpu().numpy())
drugs.index = smiles_list

In [46]:
drugs = pd.DataFrame(embeddings.cpu().numpy())
drugs.index = smiles_list

In [47]:
exp.index = exp.index.astype(str)
cellline["COSMIC_ID"] = cellline["COSMIC_ID"].astype(str)
drugs.index = drugs.index.astype(str)

In [48]:
exp_values = exp.to_numpy(dtype=np.float32)
cid_to_row = {cid: i for i, cid in enumerate(exp.index)}
drug_vec = {s: drugs.loc[s].to_numpy(dtype=np.float32) for s in drugs.index}
mask = cellline["COSMIC_ID"].isin(exp.index) & cellline["SMILES"].isin(drugs.index)
cellline_small = cellline.loc[mask].reset_index(drop=True)
print(f"usable pairs: {len(cellline_small):,} / {len(cellline):,}")

usable pairs: 284,826 / 284,826


In [49]:
cellline_vae = GeneDrugVAE(
    gene_dim, drug_dim=768, proj_dim=256, hidden=512, latent=128
).to(device)

state = torch.load("prep/models/gdsc/gdsc_best_Huber_C_max.pt", map_location=device)
cellline_vae.load_state_dict(state, strict=True)
cellline_vae.eval()

GeneDrugVAE(
  (g1): Linear(in_features=6255, out_features=512, bias=True)
  (g2): Linear(in_features=512, out_features=256, bias=True)
  (p1): Linear(in_features=768, out_features=256, bias=True)
  (p2): Linear(in_features=256, out_features=256, bias=True)
  (fc): Linear(in_features=512, out_features=512, bias=True)
  (mu): Linear(in_features=512, out_features=128, bias=True)
  (lv): Linear(in_features=512, out_features=128, bias=True)
  (d1): Linear(in_features=128, out_features=256, bias=True)
  (d2): Linear(in_features=256, out_features=6255, bias=True)
  (head): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [50]:
mp = "no"
accelerator = accelerate.Accelerator(
    mixed_precision=mp,  # None にしない
    gradient_accumulation_steps=cfg.gradient_accumulation_steps,
)

# https://github.com/huggingface/transformers/issues/26548
accelerator.dataloader_config.dispatch_batches = False

if hasattr(cfg, "force_wandb_name") and cfg.force_wandb_name:
    save_dir = cfg.save_dir.format(cfg.wandb_name)
else:
    # unknown_cfg を使わず、cfg 内の既存値だけで wandb_name を決定
    if not hasattr(cfg, "wandb_name") or cfg.wandb_name is None:
        cfg.wandb_name = "default_run"  # 任意のデフォルト名
    save_dir = cfg.save_dir.format(
        cfg.latent_dims if hasattr(cfg, "latent_dims") else cfg.wandb_name
    )

logger = Logger(
    project=cfg.wandb_project,
    name=cfg.wandb_name,
    entity="inoue0426",
    dummy=(cfg.wandb_project is None) or not (cfg.use_wandb),
    config=cfg,
)

print("Running Experiment:", cfg.wandb_name)

cc_cellline_exp_recons_cos,▆▆▅▆▆▆▆▆▆▇▆▆▆▇▆▅▄▅▅▅▅▅▅▅▄▅▆▅▄▄█▄▁▅▄▇▅▅▃▁
cc_cellline_exp_recons_rmse,█▇▇▇█▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▆▇▇▆▆▇▆▆▅▆▇▆▆▅▃▃▃▅▁▁
cc_cellline_exp_tcga_exp_trans_cos,████████▇▇▇▇▇▇▆▆▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
cc_cellline_exp_tcga_exp_trans_rmse,██████████▇▇▇▇▇▇▇▇▇▇▆▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁
cc_rec_loss,▇▇██▇▇▇█▇▇▇▇▆▆▇▆▆▆▆▆▅▆▅▅▅▅▅▄▄▄▅▅▄▅▄▄▄▅▄▁
cc_tcga_exp_cellline_exp_trans_cos,███████▇▇▇▇▇▇▇▇▇▇▆▆▆▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
cc_tcga_exp_cellline_exp_trans_rmse,█████████████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▂▂▂▂▂▁▁▁▁▁
cc_tcga_exp_recons_cos,▇▇█▆▇▇▇██▇▇▇▆▇▆▆▆▆▆▆▆▆▅▅▅▄▄▄▄▄▃▂▂▂▃▂▂▂▂▁
cc_tcga_exp_recons_rmse,▇███▇▆▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▄▃▃▄▃▃▃▂▃▁▂
cc_trans_loss,██████████▇██▇▇▇▇▇▇▇▇▇▆▆▆▆▆▅▄▃▃▂▂▂▂▁▁▁▁▁
+50,...


Running Experiment: test_bio


In [51]:
sup_encs = {cfg.sup_emb: cellline_vae}
unsup_enc = {cfg.unsup_emb: patient_vae}
encoder_dims = {cfg.sup_emb: 128}
translator = load_n_translator(cfg, encoder_dims)
unsup_dim = {cfg.unsup_emb: 128}
translator.add_encoders(unsup_dim, overwrite_embs=[cfg.unsup_emb])
translator

initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.nn.modules.linear.Linear'>
initializing <class 'torch.n

TransformTranslator(
  (in_adapters): ModuleDict(
    (cellline_exp): MLPWithResidual(
      (layers): ModuleList(
        (0): Sequential(
          (0): Linear(in_features=128, out_features=1024, bias=True)
          (1): SiLU()
        )
        (1): Sequential(
          (0): Linear(in_features=1024, out_features=1024, bias=True)
          (1): SiLU()
          (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): Dropout(p=0.1, inplace=False)
        )
        (2): Sequential(
          (0): Linear(in_features=1024, out_features=1024, bias=True)
          (1): Dropout(p=0.1, inplace=False)
          (2): SiLU()
          (3): Linear(in_features=1024, out_features=1024, bias=True)
        )
      )
    )
    (tcga_exp): MLPWithResidual(
      (layers): ModuleList(
        (0): Sequential(
          (0): Linear(in_features=128, out_features=1024, bias=True)
          (1): SiLU()
        )
        (1): Sequential(
          (0): Linear(

In [52]:
dset = Dataset.from_dict(
    {"x_gene": x_gene_tensor.numpy(), "x_drug": x_drug_tensor.numpy()}
)

Dataset({
    features: ['x_gene', 'x_drug'],
    num_rows: 10596
})


In [53]:
def select_with_replacement(dataset, n, seed=None):
    rng = np.random.RandomState(seed)
    idx = rng.randint(0, len(dataset), size=n).tolist()
    return dataset.select(idx)

In [54]:
cfg.num_params = sum(x.numel() for x in translator.parameters())
num_workers = min(get_num_proc(), 8)
# dset = load_streaming_embeddings(cfg.dataset)
print(f"Using {num_workers} workers and {len(dset)} datapoints")

dset_dict = dset.train_test_split(test_size=cfg.val_size, seed=cfg.val_dataset_seed)
dset = dset_dict["train"]
valset = dset_dict["test"]
dset.set_format(type="torch", columns=["x_gene", "x_drug"])
valset.set_format(type="torch", columns=["x_gene", "x_drug"])

supset = dset
dset = dset.shuffle(seed=cfg.train_dataset_seed)
cfg.num_points = max(len(cellline_small), len(dset))
supset = select_with_replacement(dset, n=cfg.num_points, seed=cfg.train_dataset_seed)

cellline_small = cellline_small.sample(
    n=cfg.num_points, random_state=cfg.train_dataset_seed
)

Using 8 workers and 10596 datapoints


In [57]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()
pin_memory_flag = bool(use_cuda)

sup_dataloader = DataLoader(
    supset,
    batch_size=cfg.bs,
    shuffle=True,
    num_workers=num_workers // 2,
    pin_memory=pin_memory_flag,
    drop_last=True,
)

if use_val_set:
    valloader = DataLoader(
        valset,
        batch_size=cfg.bs,
        shuffle=False,
        num_workers=num_workers // 2,
        pin_memory=pin_memory_flag,
        drop_last=False,  # 検証では端数も評価
    )

ds = GDSCDataset(
    cellline_small,
    exp_values,
    cid_to_row,
    drug_vec,
    dtype=torch.float32,
    pin_memory=True,
    copy_arrays=False,
)


def collate_xg_xd(batch):
    # batch: List[Tuple[xg, xd, y]]
    xg = torch.stack([b[0] for b in batch], 0)
    xd = torch.stack([b[1] for b in batch], 0)
    # y は使わないなら返さない（返すなら 'label': ...）
    return {"x_gene": xg, "x_drug": xd}


unsup_dataloader = DataLoader(
    ds,
    batch_size=cfg.bs,
    shuffle=True,
    num_workers=0,  # macOS ならまず 0 でOK（後で上げる）
    pin_memory=(device.type == "cuda"),
    drop_last=True,
    collate_fn=collate_xg_xd,  # ★ここがポイント
)

In [58]:
opt = torch.optim.Adam(
    translator.parameters(), lr=cfg.lr, fused=False, betas=(0.5, 0.999)
)

In [59]:
######################################################################################
disc = Discriminator(
    latent_dim=translator.in_adapters[cfg.unsup_emb].in_dim,
    discriminator_dim=cfg.disc_dim,
    depth=cfg.disc_depth,
    weight_init=cfg.weight_init,
)
disc_opt = torch.optim.Adam(
    disc.parameters(), lr=cfg.disc_lr, eps=cfg.eps, betas=(0.5, 0.999)
)

cfg.num_disc_params = sum(x.numel() for x in disc.parameters())
# print(f"Number of discriminator parameters:", cfg.num_disc_params)
######################################################################################
sup_disc = Discriminator(
    latent_dim=translator.in_adapters[cfg.sup_emb].in_dim,
    discriminator_dim=cfg.disc_dim,
    depth=cfg.disc_depth,
)
sup_disc_opt = torch.optim.Adam(
    sup_disc.parameters(), lr=cfg.disc_lr, eps=cfg.eps, betas=(0.5, 0.999)
)

cfg.num_sup_disc_params = sum(x.numel() for x in sup_disc.parameters())
# print(f"Number of supervised discriminator parameters:", cfg.num_sup_disc_params)
# print(sup_disc)
######################################################################################
latent_disc = Discriminator(
    latent_dim=cfg.d_adapter,
    discriminator_dim=cfg.disc_dim,
    depth=cfg.disc_depth,
    weight_init=cfg.weight_init,
)
latent_disc_opt = torch.optim.RMSprop(
    latent_disc.parameters(), lr=cfg.disc_lr, eps=cfg.eps
)
cfg.num_latent_disc_params = sum(x.numel() for x in latent_disc.parameters())
# print(f"Number of latent discriminator parameters:", cfg.num_latent_disc_params)
# print(latent_disc)
latent_disc_opt = torch.optim.Adam(
    latent_disc.parameters(), lr=cfg.disc_lr, eps=cfg.eps, betas=(0.5, 0.999)
)
######################################################################################
similarity_disc = Discriminator(
    latent_dim=cfg.bs,
    discriminator_dim=cfg.disc_dim,
    depth=cfg.disc_depth,
    weight_init=cfg.weight_init,
)
similarity_disc_opt = torch.optim.RMSprop(
    similarity_disc.parameters(), lr=cfg.disc_lr, eps=cfg.eps
)
cfg.num_similarity_disc_params = sum(x.numel() for x in similarity_disc.parameters())
# print(f"Number of similarity discriminator parameters:", cfg.num_similarity_disc_params)
# print(similarity_disc)
similarity_disc_opt = torch.optim.Adam(
    similarity_disc.parameters(), lr=cfg.disc_lr, eps=cfg.eps, betas=(0.5, 0.999)
)
######################################################################################

In [60]:
max_num_epochs = int(np.ceil(cfg.epochs))
steps_per_epoch = len(supset) // cfg.bs
total_steps = steps_per_epoch * cfg.epochs / cfg.gradient_accumulation_steps
warmup_length = cfg.warmup_length if hasattr(cfg, "warmup_length") else 100


def lr_lambda(step):
    if step < warmup_length:
        return min(1, step / warmup_length)
    else:
        if hasattr(cfg, "no_scheduler") and cfg.no_scheduler:
            return 1
        return 1 - (step - warmup_length) / max(1, total_steps - warmup_length)


scheduler = LambdaLR(opt, lr_lambda=lr_lambda)
disc_scheduler = LambdaLR(disc_opt, lr_lambda=lr_lambda)
sup_disc_scheduler = LambdaLR(sup_disc_opt, lr_lambda=lr_lambda)
latent_disc_scheduler = LambdaLR(latent_disc_opt, lr_lambda=lr_lambda)
similarity_disc_scheduler = LambdaLR(similarity_disc_opt, lr_lambda=lr_lambda)

if cfg.finetune_mode:
    assert hasattr(cfg, "load_dir")
    print(f"Loading models from {cfg.load_dir}...")
    translator.load_state_dict(
        torch.load(cfg.load_dir + "model.pt", map_location="cpu"), strict=False
    )
    disc.load_state_dict(torch.load(cfg.load_dir + "disc.pt", map_location="cpu"))

translator, opt, scheduler = accelerator.prepare(translator, opt, scheduler)
disc, disc_opt, disc_scheduler = accelerator.prepare(disc, disc_opt, disc_scheduler)
sup_disc, sup_disc_opt, sup_disc_scheduler = accelerator.prepare(
    sup_disc, sup_disc_opt, sup_disc_scheduler
)
latent_disc, latent_disc_opt, latent_disc_scheduler = accelerator.prepare(
    latent_disc, latent_disc_opt, latent_disc_scheduler
)
similarity_disc, similarity_disc_opt, similarity_disc_scheduler = accelerator.prepare(
    similarity_disc, similarity_disc_opt, similarity_disc_scheduler
)
sup_dataloader, unsup_dataloader = accelerator.prepare(sup_dataloader, unsup_dataloader)

In [61]:
if cfg.gan_style == "vanilla":
    gan_cls = VanillaGAN
elif cfg.gan_style == "least_squares":
    gan_cls = LeastSquaresGAN
elif cfg.gan_style == "relativistic":
    gan_cls = RelativisticGAN
else:
    raise ValueError(f"Unknown GAN style: {cfg.gan_style}")

latent_gan = gan_cls(
    cfg=cfg,
    generator=translator,
    discriminator=latent_disc,
    discriminator_opt=latent_disc_opt,
    discriminator_scheduler=latent_disc_scheduler,
    accelerator=accelerator,
)
similarity_gan = gan_cls(
    cfg=cfg,
    generator=translator,
    discriminator=similarity_disc,
    discriminator_opt=similarity_disc_opt,
    discriminator_scheduler=similarity_disc_scheduler,
    accelerator=accelerator,
)
gan = gan_cls(
    cfg=cfg,
    generator=translator,
    discriminator=disc,
    discriminator_opt=disc_opt,
    discriminator_scheduler=disc_scheduler,
    accelerator=accelerator,
)
sup_gan = gan_cls(
    cfg=cfg,
    generator=translator,
    discriminator=sup_disc,
    discriminator_opt=sup_disc_opt,
    discriminator_scheduler=sup_disc_scheduler,
    accelerator=accelerator,
)

sup_iter = None
if hasattr(cfg, "unsup_points"):
    sup_iter = iter(sup_dataloader)

if hasattr(cfg, "val_size") and hasattr(cfg, "patience") and hasattr(cfg, "min_delta"):
    early_stopper = EarlyStopper(
        patience=cfg.patience, min_delta=cfg.min_delta, increase=False
    )
    early_stopping = True
else:
    early_stopping = False

In [62]:
import re


def print_val_summary(epoch, val_res, sup_name=None, unsup_name=None, compact=True):
    """
    epoch: int
    val_res: dict[str, float]
    sup_name / unsup_name: optional domain names
    compact=True で簡略出力（おすすめ）
    """

    # --- ドメイン名推定 ---
    if sup_name is None or unsup_name is None:
        rec_flags = [
            m.group(1)
            for k in val_res.keys()
            if (m := re.match(r"^val/rec_(.+?)_cos$", k))
        ]
        rec_flags = list(dict.fromkeys(rec_flags))
        if len(rec_flags) >= 2:
            sup_name, unsup_name = rec_flags[:2]
        else:
            edges = [
                m.groups()
                for k in val_res.keys()
                if (m := re.match(r"^val/([^_/]+)_([^_/]+)_cos$", k))
            ]
            if edges:
                sup_name, unsup_name = edges[0]
            else:
                sup_name = sup_name or "sup"
                unsup_name = unsup_name or "unsup"

    def getf(key):
        v = val_res.get(key)
        return float(v) if v is not None else None

    # --- 値取得 ---
    rec_sup = getf(f"val/rec_{sup_name}_cos")
    rec_uns = getf(f"val/rec_{unsup_name}_cos")

    cos_su, vsp_su, mse_su = (
        getf(f"val/{sup_name}_{unsup_name}_cos"),
        getf(f"val/{sup_name}_{unsup_name}_vsp"),
        getf(f"val/{sup_name}_{unsup_name}_mse"),
    )
    cos_us, vsp_us, mse_us = (
        getf(f"val/{unsup_name}_{sup_name}_cos"),
        getf(f"val/{unsup_name}_{sup_name}_vsp"),
        getf(f"val/{unsup_name}_{sup_name}_mse"),
    )

    # --- 出力 ---
    print(f"\n📊 Epoch {epoch + 1} Validation Summary")
    print("──────────────────────────────────────────")

    # reconstruction
    if rec_sup is not None or rec_uns is not None:
        print(f"  🔹 Self-reconstruction cosine:")
        if rec_sup is not None:
            print(f"    {sup_name:>12s}: {rec_sup: .4f}")
        if rec_uns is not None:
            print(f"    {unsup_name:>12s}: {rec_uns: .4f}")

    # translation
    print(
        f"  🔹 {sup_name} → {unsup_name}: cos={cos_su: .4f}, vsp={vsp_su: .4f}, mse={mse_su: .6f}"
    )
    print(
        f"  🔹 {unsup_name} → {sup_name}: cos={cos_us: .4f}, vsp={vsp_us: .4f}, mse={mse_us: .6f}"
    )

    if not compact:
        # 変動指標も出したい場合のみ
        for src, tgt in [(sup_name, unsup_name), (unsup_name, sup_name)]:
            std, cos_var, vsp_var, cos_se, vsp_se = (
                getf(f"val/{src}_{tgt}_std"),
                getf(f"val/{src}_{tgt}_cos_var"),
                getf(f"val/{src}_{tgt}_vsp_var"),
                getf(f"val/{src}_{tgt}_cos_se"),
                getf(f"val/{src}_{tgt}_vsp_se"),
            )
            if any(x is not None for x in [std, cos_var, vsp_var, cos_se, vsp_se]):
                print(
                    f"    ({src}→{tgt}) Var: std={std: .4f}, cosVar={cos_var: .4f}, vspVar={vsp_var: .4f}"
                )

    print("===========================================")

In [63]:
print("len(sup_dataloader)  =", len(sup_dataloader))
print("len(unsup_dataloader)=", len(unsup_dataloader))
print("len(valloader)=", len(valloader))
print("cfg.bs=", cfg.bs)

for epoch in range(max_num_epochs):
    print(f"\n===== Epoch {epoch + 1}/{max_num_epochs} =====")

    # --- Validation ---
    if use_val_set:
        print("[Eval] Running validation...")
        with torch.no_grad(), accelerator.autocast():
            translator.eval()
            val_res = {}
            recons, trans, heatmap_dict, _, _, _ = eval_loop_(
                cfg,
                translator,
                {**sup_encs, **unsup_enc},
                valloader,
                device=accelerator.device,
            )

            print(f"[Eval] Validation finished. Processing results...")
            for flag, res in recons.items():
                for k, v in res.items():
                    if k == "cos":
                        val_res[f"val/rec_{flag}_{k}"] = v

            for target_flag, d in trans.items():
                for flag, res in d.items():
                    for k, v in res.items():
                        if flag == cfg.unsup_emb and target_flag == cfg.unsup_emb:
                            continue
                        val_res[f"val/{flag}_{target_flag}_{k}"] = v

            if len(heatmap_dict) > 0:
                print(f"[Eval] Found {len(heatmap_dict)} heatmap metrics.")
                for k, v in heatmap_dict.items():
                    if "heatmap" in k and "top" not in k:
                        val_res[f"val/{k}"] = v
                    else:
                        val_res[f"val/{k} (avg. {cfg.top_k_batches} batches)"] = v

            translator.train()
            print("[Eval] Validation metrics collected.")

        # --- Early Stopping ---
        if epoch >= getattr(cfg, "min_epochs", 0) and early_stopping:
            score_values = [v for k, v in val_res.items() if "top_rank" in k]
            score = np.mean(score_values) if len(score_values) > 0 else 0.0
            print(f"[Eval] Validation score (mean top_rank): {score:.4f}")

            if early_stopper.early_stop(score):
                print("🛑 Early stopping triggered!")
                break
            if early_stopper.counter == 0 and score < early_stopper.opt_val:
                print(
                    f"[Eval] Saving model — new best score ({score:.4f} < {early_stopper.opt_val:.4f})"
                )
                save_everything(
                    cfg,
                    translator,
                    opt,
                    [gan, sup_gan, latent_gan, similarity_gan],
                    save_dir,
                )

    print_val_summary(epoch, val_res)

    # --- Training ---
    max_num_batches = None
    print(f"[Train] Starting training loop for epoch {epoch + 1}")
    if epoch + 1 >= max_num_epochs:
        max_num_batches = max(1, int((cfg.epochs - epoch) * len(supset) // cfg.bs))
        print(
            f"[Train] Final epoch detected. Setting max_num_batches = {max_num_batches}"
        )

    sup_iter = training_loop_(
        save_dir=save_dir,
        accelerator=accelerator,
        translator=translator,
        gan=gan,
        sup_gan=sup_gan,
        latent_gan=latent_gan,
        similarity_gan=similarity_gan,
        sup_dataloader=sup_dataloader,
        sup_iter=sup_iter,
        unsup_dataloader=unsup_dataloader,
        sup_encs=sup_encs,
        unsup_enc=unsup_enc,
        cfg=cfg,
        opt=opt,
        scheduler=scheduler,
        logger=logger,
        max_num_batches=max_num_batches,
    )

    print(f"[Train] Epoch {epoch + 1} completed.")
    print("-----------------------------------------")

len(sup_dataloader)  = 1112
len(unsup_dataloader)= 1112
len(unsup_dataloader)= 9
cfg.bs= 256

===== Epoch 1/50 =====
[Eval] Running validation...
[Eval] Validation finished. Processing results...
[Eval] Found 18 heatmap metrics.
[Eval] Validation metrics collected.

📊 Epoch 1 Validation Summary
──────────────────────────────────────────
  🔹 Self-reconstruction cosine:
    cellline_exp:  0.0966
        tcga_exp:  0.0971
  🔹 cellline_exp → tcga_exp: cos= 0.0130, vsp= 0.6236, mse= 0.015422
  🔹 tcga_exp → cellline_exp: cos=-0.0096, vsp= 0.7803, mse= 0.015775
[Train] Starting training loop for epoch 1


Training: 100%|█| 1112/1112 [07:18<00:00,  2.54it/s, rec_loss=0.0982, reverse_rec_loss=0.876, vsp_loss=0.117, cc_vsp_loss=0.143, cc_rec_loss

[Train] Epoch 1 completed.
-----------------------------------------

===== Epoch 2/50 =====
[Eval] Running validation...


[Eval] Validation finished. Processing results...
[Eval] Found 18 heatmap metrics.
[Eval] Validation metrics collected.

📊 Epoch 2 Validation Summary
──────────────────────────────────────────
  🔹 Self-reconstruction cosine:
    cellline_exp:  0.9216
        tcga_exp:  0.8294
  🔹 cellline_exp → tcga_exp: cos= 0.0256, vsp= 0.6074, mse= 0.015224
  🔹 tcga_exp → cellline_exp: cos= 0.0887, vsp= 0.8408, mse= 0.014239
[Train] Starting training loop for epoch 2


Training: 100%|█| 1112/1112 [07:47<00:00,  2.38it/s, rec_loss=0.046, reverse_rec_loss=0.894, vsp_loss=0.0938, cc_vsp_loss=0.134, cc_rec_loss

...

[Train] Epoch 49 completed.
-----------------------------------------

===== Epoch 50/50 =====
[Eval] Running validation...



python(82755) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(82756) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(82757) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(82759) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


[Eval] Validation finished. Processing results...
[Eval] Found 18 heatmap metrics.
[Eval] Validation metrics collected.

📊 Epoch 50 Validation Summary
──────────────────────────────────────────
  🔹 Self-reconstruction cosine:
    cellline_exp:  0.9904
        tcga_exp:  0.9735
  🔹 cellline_exp → tcga_exp: cos= 0.0536, vsp= 0.5774, mse= 0.014788
  🔹 tcga_exp → cellline_exp: cos= 0.0705, vsp= 0.8461, mse= 0.014523
[Train] Starting training loop for epoch 50
[Train] Final epoch detected. Setting max_num_batches = 1112


Training:   0%|                                                                                                    | 0/1112 [00:00<?, ?it/s]python(82769) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(82771) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(82772) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(82773) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Training: 100%|█| 1112/1112 [07:50<00:00,  2.37it/s, rec_loss=0.0068, reverse_rec_loss=0.973, vsp_loss=0.0446, cc_vsp_loss=0.064, cc_rec_los


[Train] Epoch 50 completed.
-----------------------------------------


In [64]:
val_res

{'val/rec_cellline_exp_cos': np.float64(0.9904457926750183),
 'val/rec_tcga_exp_cos': np.float64(0.9735424518585205),
 'val/tcga_exp_cellline_exp_mse': np.float64(0.014523361499110857),
 'val/tcga_exp_cellline_exp_cos': np.float64(0.0705049228337076),
 'val/tcga_exp_cellline_exp_std': np.float64(0.06716008649932013),
 'val/tcga_exp_cellline_exp_vsp': np.float64(0.8461155560281541),
 'val/tcga_exp_cellline_exp_cos_var': np.float64(0.5111868646409776),
 'val/tcga_exp_cellline_exp_vsp_var': np.float64(0.3853442668914795),
 'val/tcga_exp_cellline_exp_cos_se': np.float64(0.0074476388689522464),
 'val/tcga_exp_cellline_exp_vsp_se': np.float64(0.006466260817472408),
 'val/cellline_exp_tcga_exp_mse': np.float64(0.014788041719131999),
 'val/cellline_exp_tcga_exp_cos': np.float64(0.05356538751059108),
 'val/cellline_exp_tcga_exp_std': np.float64(0.0826455992129114),
 'val/cellline_exp_tcga_exp_vsp': np.float64(0.5774012870258756),
 'val/cellline_exp_tcga_exp_cos_var': np.float64(0.22430828379260